In [1]:
from pathlib import Path

import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import qiime2
from q2_composition import DataLoafPackageDirFmt

In [2]:
def plot_differentials(category, df, output_dir,
                       q_threshold=1.0, 
                       lfc_abs_threshold=0, 
                       sort_by='lfc', 
                       feature_ids=None,
                       figsize=(25, 25)):

    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True) 
    fig_fp = output_dir / Path(f'{category}-ancombc-barplot.pdf')
    
    if feature_ids is not None:
        df = df[df['feature-id'] in feature_ids]
    
    df = df[df['q_val'] <= q_threshold]
    df = df[np.abs(df['lfc']) >= lfc_abs_threshold]
    df = df.sort_values(sort_by)
    
    sns.set_theme(style="whitegrid")
    
    f, ax = plt.subplots(figsize=figsize)

    sns.barplot(
        data=df, x="lfc", y="feature-id", xerr=df['se'],
        capsize=.4, errcolor=".5",
        linewidth=1, edgecolor=".5", facecolor=(0, 0, 0, 0),
    )

    ax.set_title(f'{category} enrichment relative to reference')
    ax.set_xlabel('Log fold change (LFC)')
    ax.set_ylabel('Feature ID')
    
    f.savefig(fig_fp, bbox_inches='tight')

In [3]:
loaf = qiime2.Artifact.load('./ancombc-body_habitat.qza')
slices = loaf.view(DataLoafPackageDirFmt)
data = {}
for e in slices.data_slices.iter_views(pd.DataFrame):
    data[str(e[0]).replace('_slice.csv','')] = e[1]
columns = [e for e in data['lfc'].columns if e not in '(Intercept)']

figure_data = []
for category in columns:
    if category == 'id':
        continue
    df = pd.concat([data['lfc']['id'], data['lfc'][category], data['se'][category], data['q_val'][category]],
                   keys=['feature-id', 'lfc', 'se', 'q_val'], axis=1)
    figure_data.append((category, df))
    plot_differentials(category, df, './da-plots/', q_threshold=0.001)